In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import imodelsx.process_results
import sys
from ridge_utils.features import qa_questions
import numpy as np
sys.path.append('../experiments/')
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/single_question_may6'
experiment_filename = '../experiments/04_fit_single_question.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)

# get the breakdown of data in these groups
r.groupby(cols_varied).size()

In [ ]:
mets = [c for c in r.columns if 'corrs' in c and (
        'mean' in c or 'frac' in c)]
(
    r.groupby(cols_varied)[mets].mean()
    .style
    .background_gradient(
        cmap='viridis', axis=0)
    .format("{:.4f}")
)

In [ ]:
row = r.sort_values(by='corrs_test_selected_mean', ascending=False).iloc[0]

In [ ]:
qs = qa_questions.get_questions(version='v3_boostexamples', full=True)
qs_selected = row['qs_selected']
qs_selected = qs_selected % len(qs)
qs_selected_counts = pd.Series(qs_selected).value_counts()
qs_selected_frac = qs_selected_counts / qs_selected_counts.sum()

In [ ]:
plt.figure(figsize=(12, 3))
plt.bar(np.arange(len(qs_selected_frac)), np.cumsum(qs_selected_frac.values))

In [ ]:
# most popular questions
np.array(qs)[qs_selected_frac.index[:10]]

In [ ]:
plt.hist(row['corrs_test_selected'], bins=100)
plt.show()

### Only look at well-predicted voxels

In [ ]:
voxels_top = row['corrs_test_selected'] > np.percentile(
    row['corrs_test_selected'], 90)
qs_selected_top = np.array(qs)[qs_selected][voxels_top]
qs_selected_top_counts = pd.Series(qs_selected_top).value_counts()
qs_selected_top_frac = qs_selected_top_counts / qs_selected_top_counts.sum()

In [ ]:
# most popular questions
np.array(qs)[qs_selected_frac.index[:10]]